# Nettoyage des Données eBay - Graphics Cards

Ce notebook a pour objectif de nettoyer les fichiers CSV bruts extraits d’eBay pour la catégorie **graphics_cards**.
Les opérations réalisées incluent :

- La définition des chemins des dossiers de données brutes et nettoyées.
- L’application de plusieurs fonctions de nettoyage pour homogénéiser et corriger les données (titre, marque, prix, taille mémoire, etc.).
- La suppression des doublons (en conservant la ligne avec le prix minimum) et le filtrage des enregistrements valides.
- La sauvegarde des données nettoyées dans un nouveau fichier CSV.

Chaque fonction utilisée est expliquée en détail.


In [ ]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import process
from pathlib import Path


## Définition des Chemins de Données

Nous utilisons la librairie `pathlib` pour définir les chemins relatifs vers :
- Le dossier contenant les fichiers CSV bruts.
- Le dossier dans lequel seront sauvegardés les fichiers nettoyés.

Ces chemins sont définis en fonction du répertoire racine du projet.


In [ ]:
# Pour un notebook, vous pouvez définir BASE_DIR manuellement, par exemple :
BASE_DIR = Path.cwd()  # Utilise le répertoire courant comme racine du projet

# Si votre arborescence est différente, adaptez ces chemins :
RAW_DATA_DIR = BASE_DIR / 'data' / 'raw' / 'ebay' / 'graphics_cards'
CLEANED_DATA_DIR = BASE_DIR / 'data' / 'cleaned' / 'ebay' / 'graphics_cards'

# Création du dossier de données nettoyées s'il n'existe pas
CLEANED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Affichage des chemins pour vérification
print(f"Base directory: {BASE_DIR}")
print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Cleaned data directory: {CLEANED_DATA_DIR}")

# Vérifie que le dossier de données brutes existe
if not RAW_DATA_DIR.exists():
    raise FileNotFoundError(f"Raw data directory not found: {RAW_DATA_DIR}")

# Vérifie s'il y a des fichiers CSV à traiter
files = list(RAW_DATA_DIR.glob('*.csv'))
if not files:
    print(f"No CSV files found in {RAW_DATA_DIR}")
else:
    print(f"Found {len(files)} CSV files to process")


## Fonction `clean_title`

La fonction `clean_title` prend en entrée une ligne (row) d'un DataFrame et réalise les opérations suivantes :

1. Récupère les informations du modèle GPU et de la marque.
2. Définit une liste de termes indésirables (ex. "new", "used", "gpu", "hdmi", etc.) à supprimer du titre.
3. Nettoie le titre en supprimant ces termes et en éliminant les caractères spéciaux ainsi que les espaces multiples.
4. Retourne un titre nettoyé qui privilégie l'inclusion du modèle GPU et/ou de la marque si présents dans le titre d'origine.


In [ ]:
def clean_title(row):
    # Utilise le modèle GPU et la marque comme base
    gpu_model = str(row["Chipset/GPU Model"]).strip()
    brand = str(row["Brand"]).strip()

    # Liste des termes indésirables à supprimer
    unwanted_terms = [
        r'\bnew\b', r'\bused\b', r'\bgraphics\b', r'\bcard\b', r'\bgpu\b', r'\bvideo\b', r'\bhdmi\b', r'\bvga\b',
        r'\bdvi\b', r'\bdisplayport\b', r'\bminidisplayport\b', r'\busb-c\b', r'\boc\b', r'\bgddr5\b', r'\bgddr6\b',
        r'\bgddr6x\b', r'\b256-bit\b', r'\b128-bit\b', r'\b512mb\b', r'\b1gb\b', r'\b2gb\b', r'\b4gb\b', r'\b8gb\b',
        r'\b12gb\b', r'\b16gb\b', r'\b24gb\b', r'\b32gb\b', r'\b64gb\b', r'\b128mb\b', r'\b256mb\b', r'\b512mb\b',
        r'\b1yr\b', r'\bwarranty\b', r'\bfast\b', r'\bship\b', r'\btested\b', r'\bworking\b', r'\bnot working\b',
        r'\bexcellent\b', r'\bcondition\b', r'\brefurbished\b', r'\bgrade\b', r'\bwith box\b', r'\bwithout box\b',
        r'\bbulk\b', r'\boem\b', r'\bretail\b', r'\bpackage\b', r'\bpackaging\b', r'\bmodel\b', r'\bseries\b',
        r'\bversion\b', r'\bgen\b', r'\bgen\b', r'\bpcie\b', r'\bexpress\b', r'\bslot\b', r'\bconnector\b',
        r'\binterface\b', r'\bdual\b', r'\bsingle\b', r'\btriple\b', r'\bquad\b', r'\bhex\b', r'\boct\b', r'\bcore\b',
    ]

    # Nettoie le titre original en minuscules
    clean_name = row['Title'].lower()
    for term in unwanted_terms:
        clean_name = re.sub(term, "", clean_name)

    # Supprime les caractères spéciaux et les espaces multiples
    clean_name = re.sub(r"[^a-zA-Z0-9\s]", "", clean_name).strip()
    clean_name = re.sub(r"\s+", " ", clean_name)

    # Inclut le modèle GPU et la marque si présents dans le titre
    if gpu_model.lower() in clean_name and brand.lower() in clean_name:
        return f"{brand} {gpu_model}"
    elif gpu_model.lower() in clean_name:
        return gpu_model
    elif brand.lower() in clean_name:
        return f"{brand} {clean_name}"
    else:
        return f"{brand} {gpu_model}" if brand != "nan" else gpu_model


## Fonction `correct_brands`

Cette fonction corrige les noms de marque en utilisant un algorithme de correspondance floue (fuzzy matching) grâce à la librairie `fuzzywuzzy`.

Pour chaque marque présente dans la colonne `Brand` du DataFrame, la fonction :
- Compare la valeur actuelle avec la liste des marques uniques.
- Si le score de correspondance est supérieur à 85, la marque est remplacée par la correspondance trouvée.
- Retourne le DataFrame avec la colonne `Brand` mise à jour.


In [ ]:
def correct_brands(df):
    brands = df['Brand'].dropna().unique().tolist()
    brand_mapping = {}
    for brand in df['Brand']:
        if pd.isna(brand):
            continue
        match, score = process.extractOne(brand, brands)
        brand_mapping[brand] = match if score > 85 else brand
    df['Brand'] = df['Brand'].replace(brand_mapping)
    return df


## Fonction `clean_price`

La fonction `clean_price` a pour but de normaliser les valeurs de prix en :
- Supprimant tous les caractères non numériques (sauf la virgule et le point).
- Gérant les formats décimaux (par exemple, en remplaçant correctement les séparateurs).
- Convertissant la chaîne nettoyée en type `float`.

Si le prix est vide ou non défini, la fonction retourne `None`.


In [ ]:
def clean_price(price):
    if isinstance(price, str):
        price = re.sub(r'[^\d.,]', '', price)
        if ',' in price and '.' in price:
            if price.index(',') < price.index('.'):
                price = price.replace(',', '')
            else:
                price = price.replace('.', '').replace(',', '.')
        else:
            price = price.replace(',', '.')
    return float(price) if price else None


## Fonction `convert_to_gb`

Cette fonction normalise les valeurs de taille mémoire en :
- Convertissant les valeurs en MB en GB (en divisant par 1024).
- Extrayant la partie numérique d'une chaîne contenant la taille mémoire.
- Retourne la taille mémoire sous forme de `float` ou `None` si la valeur est absente.


In [ ]:
def convert_to_gb(value):
    if pd.isna(value) or value.strip() == '':
        return None
    try:
        if 'MB' in value.upper():
            return float(value.upper().replace('MB', '').strip()) / 1024
        return float(re.sub(r'[^\d.]', '', value))
    except ValueError

    ## Fonctions de Remplissage des Valeurs Manquantes

Deux fonctions sont définies pour traiter les valeurs manquantes :
- **fill_with_median_or_default** : Remplit les valeurs manquantes d'une série avec la médiane, ou avec 0 si la série est vide.
- **fill_with_mode** : Remplit les valeurs manquantes d'une série avec la valeur la plus fréquente (mode).


In [ ]:
def fill_with_median_or_default(series):
    if series.notna().any():
        return series.fillna(series.median())
    else:
        return series.fillna(0)

def fill_with_mode(series):
    if series.notna().any():
        return series.fillna(series.mode()[0])
    else:
        return series


## Fonction `remove_duplicates_with_min_price`

Cette fonction supprime les doublons du DataFrame en gardant pour chaque groupe (défini par `Brand`, `Memory Size`, `Memory Type`, `Chipset/GPU Model`) la ligne ayant le prix minimum.


In [ ]:
def remove_duplicates_with_min_price(df):
    columns_for_duplicates = ['Brand', 'Memory Size', 'Memory Type', 'Chipset/GPU Model']
    idx_min_price = df.groupby(columns_for_duplicates)['Price'].idxmin()
    return df.loc[idx_min_price].reset_index(drop=True)


## Fonction `drop_unnecessary_columns`

Cette fonction conserve uniquement les colonnes essentielles pour l’analyse finale, à savoir :
- `title`, `Price`, `Brand`, `Memory Size`, `Memory Type`, `Chipset/GPU Model`, `Connectors`, et `Collection Date`.


In [ ]:
def drop_unnecessary_columns(df):
    columns_to_keep = ['title', 'Price', 'Brand', 'Memory Size', 'Memory Type', 'Chipset/GPU Model', 'Connectors', 'Collection Date']
    existing_columns = [col for col in columns_to_keep if col in df.columns]
    return df[existing_columns]


## Traitement Global des Fichiers CSV

Dans cette partie, nous parcourons tous les fichiers CSV du dossier de données brutes et appliquons l’ensemble des fonctions de nettoyage :

1. Chargement de chaque fichier CSV dans un DataFrame.
2. Application de la fonction `clean_title` pour créer une colonne `Cleaned Title`.
3. Correction des marques via `correct_brands`.
4. Normalisation des colonnes `Price` et `Memory Size` avec les fonctions `clean_price` et `convert_to_gb`.
5. Remplissage des valeurs manquantes pour `Memory Size`, `Price`, `Memory Type` et `Connectors` par regroupement sur `Chipset/GPU Model`.
6. Suppression des doublons en conservant le produit au prix minimum.
7. Filtrage des enregistrements avec des valeurs de `Price` et `Memory Size` valides.
8. Suppression de la colonne d’origine `Title` et renommage de `Cleaned Title` en `title`.
9. Conservation uniquement des colonnes nécessaires.
10. Sauvegarde du DataFrame nettoyé dans un nouveau fichier CSV dans le dossier de données nettoyées.


In [ ]:
try:
    for file in RAW_DATA_DIR.glob('*.csv'):
        print(f"Processing file: {file}")
        df = pd.read_csv(file)
        print(f"Loaded {len(df)} rows from {file.name}")
        print(f"Columns in the file: {df.columns.tolist()}")

        # Application des fonctions de nettoyage
        df['Cleaned Title'] = df.apply(clean_title, axis=1)
        df = correct_brands(df)
        df['Price'] = df['Price'].apply(clean_price)
        df['Memory Size'] = df['Memory Size'].astype(str).apply(convert_to_gb)
        df['Memory Size'] = df.groupby('Chipset/GPU Model')['Memory Size'].transform(fill_with_median_or_default)
        df['Price'] = df.groupby('Chipset/GPU Model')['Price'].transform(fill_with_median_or_default)
        df['Memory Type'] = df.groupby('Chipset/GPU Model')['Memory Type'].transform(fill_with_mode)
        df['Connectors'] = df.groupby('Chipset/GPU Model')['Connectors'].transform(fill_with_mode)
        df = remove_duplicates_with_min_price(df)

        # Validation des données (prix et taille mémoire strictement positifs)
        df = df[(df['Price'] > 0) & (df['Memory Size'] > 0.1)]

        # Suppression de la colonne d'origine 'Title'
        if 'Title' in df.columns:
            df.drop(columns=['Title'], inplace=True)

        # Renommage de 'Cleaned Title' en 'title'


        # Conservation uniquement des colonnes utiles
        df = drop_unnecessary_columns(df)

        # Sauvegarde du DataFrame nettoyé dans un nouveau fichier CSV
        output_filename = CLEANED_DATA_DIR / f"{file.stem}_cleaned.csv"
        df.to_csv(output_filename, index=False)
        print(f"Cleaned data saved to {output_filename}")
except Exception as e:
    print(f"An error occurred: {e}")


## Conclusion

Ce notebook a permis de traiter et de nettoyer les fichiers CSV bruts extraits d’eBay pour la catégorie **graphics_cards**.
Les principales opérations effectuées sont :
- Nettoyage et homogénéisation des titres.
- Correction des noms de marques grâce à une correspondance floue.
- Normalisation des prix et de la taille mémoire.
- Remplissage des valeurs manquantes, suppression des doublons et filtrage des enregistrements non valides.
- Sauvegarde des données nettoyées dans un nouveau dossier.



Voila le result apres le nettoyage

In [3]:
import pandas as pd
from pathlib import Path

# Définir le chemin vers le fichier CSV nettoyé
csv_file_path = Path(r'C:\Users\AdMin\Desktop\ecommerce_scraper\data\cleaned\ebay\graphics_cards\graphics_cards_2025_01_29_scrape1_cleaned.csv')

df_cleaned = pd.read_csv(csv_file_path)

# Set the option to display all rows
pd.set_option('display.max_rows', None)

# Display all rows of the DataFrame
df_cleaned

,title,Price,Brand,Memory Size,Memory Type,Chipset/GPU Model,Connectors,Collection Date
0,AMD AMD FirePro W2100,8.00,AMD,2.00,DDR3,AMD FirePro W2100,DisplayPort,1/29/2025 22:26
1,AMD AMD Radeon R5 340X,23.00,AMD,2.00,GDDR3,AMD Radeon R5 340X,DisplayPort,1/29/2025 22:26
2,AMD AMD Radeon E9173,21.98,AMD,2.00,GDDR5,AMD Radeon E9173,NaN,1/29/2025 22:27
3,AMD W4100,26.26,AMD,2.00,GDDR5,W4100,Mini DisplayPort,1/29/2025 22:26
4,AMD AMD Radeon RX 5700,120.00,AMD,8.00,GDDR6,AMD Radeon RX 5700,"DVI, HDMI",1/29/2025 22:26
5,AMD AMD Radeon Instinct MI50,165.00,AMD,16.00,HBM2,AMD Radeon Instinct MI50,NaN,1/29/2025 22:26
6,ASRock asrock md radeon rx 5700 xt rx5700xt fa...,120.00,ASRock,8.00,GDDR6,AMD Radeon RX 5700 XT,DisplayPort,1/29/2025 22:26
7,ASRock asrock radeon rx 6600 pci 40 cld 8g,198.99,ASRock,8.00,GDDR6,AMD Radeon RX 6600,"DisplayPort, HDMI",1/29/2025 22:26
8,ASRock Radeon RX 6600,209.49,ASRock,8.00,GDDR6,Radeon RX 6600,"HDMI, DisplayPort",1/29/2025 22:26
9,ASUS asus geforce gtx 650 grp240504,39.00,ASUS,1.00,GDDR5,NVIDIA GeForce GTX 650,NaN,1/29/2025 22:26
